# 5.3 延后初始化
- **目录**
  - 5.3.1 延后初始化网络参数

- 到目前为止，我们忽略了建立网络时需要做的以下这些事情：
  * 我们定义了网络架构，但**没有指定输入维度**。
  * 我们添加层时**没有指定前一层的输出维度**。
  * 我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。
- 诀窍是框架的**延后初始化**（defers initialization），即直到数据第一次通过模型传递时，框架才会**动态地推断出每个层的大小**。
- 在以后，当使用卷积神经网络时，由于输入维度（即图像的分辨率）将影响每个后续层的维数，有了该技术将更加方便。
- 现在我们在编写代码时无须知道维度是什么就可以设置参数，这种能力可以大大简化定义和修改模型的任务。
- 接下来将更深入地研究初始化机制。

## 5.3.1 延后初始化网络参数
- 自定义一个延迟初始化网络参数的模型

In [1]:
import torch
import torch.nn as nn

class CustomLinear(nn.Module):
    def __init__(self, out_features):
        super(CustomLinear, self).__init__()
        self.out_features = out_features
        self.weight = None
        self.bias = None

    def forward(self, x):
        # 延后初始化参数
        if self.weight is None or self.bias is None:
            in_features = x.shape[-1]
            self.weight = nn.Parameter(torch.Tensor(self.out_features, in_features))
            self.bias = nn.Parameter(torch.Tensor(self.out_features))

            # 初始化权重和偏置，即在前向传播时初始化参数
            nn.init.kaiming_uniform_(self.weight, a=0, mode='fan_in', nonlinearity='relu')
            nn.init.zeros_(self.bias)

        return x @ self.weight.t() + self.bias

# 创建一个自定义线性层实例
custom_linear = CustomLinear(10)
print('前向传播前的参数值：', custom_linear.weight)
# 使用随机输入张量
X = torch.rand(2, 20)
output = custom_linear(X)
print('\n前向传播后的参数形状及其值：')
custom_linear.weight.shape,custom_linear.weight

前向传播前的参数值： None

前向传播后的参数形状及其值：


(torch.Size([10, 20]),
 Parameter containing:
 tensor([[-0.2010, -0.3867,  0.1535,  0.3602,  0.4944, -0.1069,  0.2682,  0.4317,
          -0.1254,  0.0297, -0.1777,  0.2044, -0.5152,  0.3057, -0.0886,  0.4759,
           0.3039, -0.5366,  0.4115, -0.4545],
         [ 0.1527,  0.3749, -0.3625, -0.0119,  0.3781, -0.3943, -0.0286, -0.0266,
          -0.3532,  0.5460, -0.0886,  0.1037, -0.4048, -0.1733,  0.4334,  0.3845,
           0.1415, -0.3082,  0.1364, -0.4234],
         [ 0.1889, -0.0147, -0.0064,  0.4771, -0.0843, -0.3223, -0.4457,  0.5387,
          -0.3342, -0.2725, -0.3292,  0.0328,  0.3403,  0.3548, -0.3319, -0.3659,
           0.3398,  0.4780,  0.5352,  0.4583],
         [ 0.1622,  0.0610,  0.1710, -0.2607, -0.1237,  0.2866,  0.5278, -0.0436,
          -0.1756, -0.2497,  0.3399,  0.2321, -0.2702,  0.4515, -0.2746, -0.1913,
          -0.2582,  0.0438, -0.3915, -0.3052],
         [ 0.4998,  0.3642, -0.1349, -0.2395, -0.0197, -0.0591,  0.0085,  0.4109,
           0.4158,  0.2711, 

- **说明：**
  - 在此示例中，我们定义了一个名为`CustomLinear`的自定义线性层类。它继承自`nn.Module`。在类的构造函数中，我们将输入权重和偏置设为`None`。这意味着权重和偏置参数在实例化时不会被初始化。

  - 然后，在`forward`方法中，我们检查权重和偏置是否为`None`。如果它们尚未初始化，我们根据输入张量`x`的形状来初始化它们。请注意，当我们第一次调用`forward`方法并传递输入数据时，延后初始化发生。

  - 在本示例中，我们使用了`Kaiming`均匀分布对权重进行初始化，并将偏置初始化为零。

----------
- **说明：延后初始化补充解释**
  - 概念：
    - 延后初始化是一种策略，它允许模型在没有具体输入维度信息时构建。
    - 参数的实际初始化推迟到收到第一批数据时自动进行。
  - 工作原理：
     - 模型首次接收到数据时，根据数据的具体形状推断每一层的参数维度。
     - 框架动态地分配必要的内存并初始化参数。
  - 动态调整优势：
    - 延后初始化允许模型动态适应不同的输入尺寸。
    - 减少了模型定义时的工作量和出错可能性。
  - 举例：
    - 假设我们构建一个简单的全连接神经网络，其中包含输入层、隐藏层和输出层。
    - 在不使用延后初始化的情况下，如果我们想要处理28x28和32x32两种大小的图像，我们需要为每种情况分别计算并指定隐藏层的输入维度。
    - 使用延后初始化，我们只需定义层的连接方式，不需要指定具体的输入维度。
    - 当我们第一次向网络输入28x28的图像时，框架自动推断出隐藏层的参数应该是多少。
    - 如果我们稍后决定将图像尺寸改变为32x32，框架也会在新的数据第一次通过时自动调整参数维度。
  - 框架支持：
    - 许多现代深度学习框架，如PyTorch和MXNet，支持延后初始化。这使得研究者和工程师可以更加灵活地设计和实验不同的网络结构。
  - 延后初始化是一种允许深度学习模型在不具备完整输入维度信息时构建的策略，它通过在模型第一次接收数据时自动推断参数维度，从而简化模型设计和参数初始化过程。
------------

## 小结

* 延后初始化使框架能够自动推断参数形状，使修改模型架构变得容易，避免了一些常见的错误。
* 我们可以通过模型传递数据，使框架最终初始化参数。